# Regularization of Linear Regression

본 코드는 Ridge Regularization의 구현 예시입니다.
본 코드는 0과 1 사이의 X 들을 싸인함수에 fitting할때, Regularization이 Regression 에 미치는 영향을 살펴봅니다.

## 실제 구현부

Regularization은 학습의 정확도는 조금 포기하면서, 일반적인 데이터를 충분히 잘 설명할 수 있도록 모델을 데이터에 대해 다소 둔감하게 만드는 기술입니다. 즉, 기존의 Loss Function 에 Regularization term을 더한 Loss Function을 최소화하는 매개변수를 찾습니다. 그 중 Ridge Regularization을 이용한 Loss Function은 다음과 같습니다.
$$E(w)=\frac{1}{2}\sum_{n=0}^N (train_n-g(x_n,w))^2+\frac{\lambda}{2}\lVert w \rVert ^2$$

위 Loss Function을 최소화하고자 미분계수가 0이되는 지점을 찾아 w에 대해 정리하면 다음과 같은 closed form의 w를 찾을 수 있습니다.
$$w = (X^TX+\lambda I)^{-1}X^T*train$$

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv

%matplotlib inline  

In [11]:
x = np.arange(0,1,0.01)
f = np.sin(2*np.pi*x)
g1 = 0 * x; #g1=0으로 채워진 리스트

w = np.polyfit(x,f,1); #linear regression 의 coefficient 값을 return
g2 = np.polyval(w,x); # g2 = w'x

plt.figure(1, figsize=(7,5)); 
plt.plot(x,f, color="blue"); #sin함수 plotting
plt.plot(x,g1, color="red"); #y=0 상수함수 plotting
plt.plot(x,g2, color="green"); #y=w'x 선형회귀 plotting

e1 = np.mean(np.dot((g1-f),(g1-f))); #상수함수의 RSS
e2 = np.mean(np.dot((g2-f),(g2-f))); #선형회귀의 RSS

# number of observations
D = 10;
trainingCaseG1 = np.zeros((D, 100));
trainingCaseG2 = np.zeros((D, 100));
trainingCaseG3 = np.zeros((D, 100));

for i in range(D):
    trainingCaseX = np.random.uniform(0,1,2); #0과 1사이의 수 2개를 무작위로 뽑음
    trainingCaseF = np.sin(2*np.pi*trainingCaseX); #위 무작위 x에 대한 sin함수값
    plt.plot(trainingCaseX,trainingCaseF, 'mo'); 
    
    tempTrainingCaseX = np.zeros((2,2));
    tempTrainingCaseX[0,:] = trainingCaseX;
    tempTrainingCaseX[1,:] = [1,1];
    randa = 1;
    
    # w = (X'*X+ramda*I)^(-1)*X'*trainF, linear regression 에 L2 정규화를 적용한 계수를 찾는 계산
    w_r = np.dot(np.transpose(np.dot(inv(np.dot(np.transpose(tempTrainingCaseX),tempTrainingCaseX) + randa + np.eye(2)), np.transpose(tempTrainingCaseX))), np.transpose(trainingCaseF));
    tempX = np.zeros((2,np.size(x))); #0으로 채워진 2X100 matrix
    tempX[0,:] = x;
    
    for j in range(np.size(x)): #생성한 x의 갯수만큼 반복
    tempX[1,j] = 1;
    # regularized regression line, 1X100 Matrix
    trainingCaseG3[i,:] = np.transpose(np.dot(np.transpose(w_r),tempX));  #y=w'x 하려면 x[0]=1, x[1] =x여야하지않을까?
    
    # First Degree Line
    w = np.polyfit(trainingCaseX, trainingCaseF, 1);
    trainingCaseG2[i,:] = np.polyval(w,x);

    # Zero Degree Line
    w = np.polyfit(trainingCaseX, trainingCaseF, 0);
    trainingCaseG1[i,:] = np.polyval(w,x);
    
meanG1 = np.zeros((np.size(x),1));
meanG2 = np.zeros((np.size(x),1));
meanG3 = np.zeros((np.size(x),1));

stdG1 = np.zeros((np.size(x),1));
stdG2 = np.zeros((np.size(x),1));
stdG3 = np.zeros((np.size(x),1));

biasXG1 = np.zeros((np.size(x),1));
biasXG2 = np.zeros((np.size(x),1));
biasXG3 = np.zeros((np.size(x),1));
    
for j in range(np.size(x)):
    meanG1[j] = np.mean(trainingCaseG1[:,j]);
    stdG1[j] = np.std(trainingCaseG1[:,j]);
    biasXG1[j] = (meanG1[j] - f[j])*(meanG1[j] - f[j]);
    
    meanG2[j] = np.mean(trainingCaseG2[:,j]);
    stdG2[j] = np.std(trainingCaseG2[:,j]);
    biasXG2[j] = (meanG2[j] - f[j])*(meanG2[j] - f[j]);
    
    meanG3[j] = np.mean(trainingCaseG3[:,j]);
    stdG3[j] = np.std(trainingCaseG3[:,j]);
    biasXG3[j] = (meanG3[j] - f[j])*(meanG3[j] - f[j]);
    
plt.errorbar(x,meanG3, yerr = stdG3);

deviationG1 = np.zeros((np.size(x), D));
deviationG2 = np.zeros((np.size(x), D));
deviationG3 = np.zeros((np.size(x), D));

for i in range(np.size(x)):
    for j in range(D):
        deviationG1[i,j] = (trainingCaseG1[j,i] - meanG1[i]) * (trainingCaseG1[j,i] - meanG1[i]); 
        deviationG2[i,j] = (trainingCaseG2[j,i] - meanG2[i]) * (trainingCaseG2[j,i] - meanG2[i]); 
        deviationG3[i,j] = (trainingCaseG3[j,i] - meanG3[i]) * (trainingCaseG3[j,i] - meanG3[i]); 
        
biasG1 = np.mean(biasXG1); # y=0 의 bias
biasG2 = np.mean(biasXG2); # linear regression 의 bias
biasG2 = np.mean(biasXG3); # regularized linear regression 의 bias

varG1 = np.mean(np.mean(deviationG1)); #  상수함수의 standard deviation
varG2 = np.mean(np.mean(deviationG2)); # linear regression 의 standard deviation
varG3 = np.mean(np.mean(deviationG3)); # regularized linear regression 의 standard deviation

plt.show();
print("bias of constant function is", biasG1)
print("bias of linear regression is", biasG2)
print("bias of regularized linear regression is", biasG3)
print("Variance of constant function is", varG1)
print("Variance of linear regression is", varG2)
print("Variance of regularized linear regression is", varG3)

ValueError: shapes (2,2) and (100,) not aligned: 2 (dim 1) != 100 (dim 0)